# The set of functions were written for plotting xrd diffraction patterns against simulated diffraction patterns generated from cif crystal structure files from the materials project.

In [ ]:
import os
import matplotlib.pyplot as plt
from statistics import median

def makeDirectories(baseDirectory):
    '''Setup and return paths for essential directories.'''
    xrdDataDirectory = os.path.join(baseDirectory, "xrdData")
    xrdPeakDirectory = os.path.join(baseDirectory, "xrdPeaks")
    plotsDirectory = os.path.join(baseDirectory, "xrdPlots")
    os.makedirs(plotsDirectory, exist_ok=True)
    return xrdDataDirectory, xrdPeakDirectory, plotsDirectory

def loadXrdData(filePath, xLimit):
    '''Load XRD data from a file within specified x limits.'''
    twoThetas, intensities = [], []
    encodings = ['ascii', 'utf-8-sig', 'ISO-8859-1']
    
    for encoding in encodings:
        try:
            with open(filePath, 'r', encoding=encoding) as file:
                header_skipped = False
                for line in file:
                    if not header_skipped:
                        header_skipped = True
                        continue
                    theta, intensity = map(float, line.strip().split()[:2])
                    if xLimit[0] <= theta <= xLimit[1]:
                        twoThetas.append(theta)
                        intensities.append(intensity)
            break  # Break out of the loop if reading was successful
        except UnicodeDecodeError:
            continue  # Try the next encoding if there was an error
    return twoThetas, intensities

def normalizeIntensities(intensitiesData, intensitiesPeak):
    '''Normalize both data and peak intensities to the same scale.'''
    if not intensitiesData or not intensitiesPeak:
        return intensitiesData, intensitiesPeak  # Return original if any list is empty

    maxIntensityData = max(intensitiesData)
    maxIntensityPeak = max(intensitiesPeak)

    # Avoid division by zero
    if maxIntensityPeak == 0 or maxIntensityData == 0:
        return intensitiesData, intensitiesPeak

    # Normalize intensities to the maximum of the data set
    scaledIntensitiesData = [i / maxIntensityData for i in intensitiesData]
    scaledIntensitiesPeak = np.array([(i/10) / maxIntensityPeak * maxIntensityData for i in intensitiesPeak]) * 5

    return scaledIntensitiesData, scaledIntensitiesPeak

def plotXrdDataAndPeaks(dataDirectory, peakDirectory, plotsDirectory, xLimit):
    '''Plot XRD data with overlayed peaks and save the figure.'''
    for dataFile in os.listdir(dataDirectory):
        twoThetasData, intensitiesData = loadXrdData(os.path.join(dataDirectory, dataFile), xLimit)

        for peakFile in os.listdir(peakDirectory):
            plt.figure(figsize=(18, 6))
            twoThetasPeak, intensitiesPeak = loadXrdData(os.path.join(peakDirectory, peakFile), xLimit)
            
            # Normalize intensities to make them comparable
            scaledIntensitiesData, scaledIntensitiesPeak = normalizeIntensities(intensitiesData, intensitiesPeak)
            
            # Plot normalized XRD data
            plt.plot(twoThetasData, scaledIntensitiesData, label='XRD Data', color='blue')
            # Plot peaks with normalized intensities
            plt.vlines(twoThetasPeak, ymin=0, ymax=scaledIntensitiesPeak, linewidth=2, label=f'PXRD Simulation', color="red")
            
            finalizePlot(dataFile, peakFile, plotsDirectory, xLimit)

def finalizePlot(dataFileName, peakFileName, plotsDirectory, xLimit):
    '''Finalize the plotting settings and save the plot to a file.'''
    dataFileTitle = formatFileName(dataFileName)
    peakFileTitle = formatFileName(peakFileName)
    plt.title(f"XRD Data: {dataFileTitle} | Crystal: {peakFileTitle}")
    plt.xlabel('2θ [degrees]')
    plt.ylabel('Intensity [a.u.]')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.xlim(xLimit)
    plt.ylim(0,1)
    plotFileName = f"{dataFileTitle}_with_{peakFileTitle}_plot.png".replace(' ', '_')
    plt.savefig(os.path.join(plotsDirectory, plotFileName))
    plt.close()  # Close the figure to free up memory
    print(f"Plot saved as {plotFileName}")

def formatFileName(fileName):
    '''Format the filename to a more readable version for titles.'''
    return fileName.replace('_xrd.txt', '').replace('_', ' ').title()

def xrdPlotter(directory, xLimit):
    '''Entry function to set up plot generation for XRD data and peaks.'''
    xrdDataDirectory, xrdPeakDirectory, plotsDirectory = makeDirectories(directory)
    if not os.path.exists(xrdDataDirectory) or not os.path.exists(xrdPeakDirectory):
        print("The specified directories don't exist.")
        return
    
    plotXrdDataAndPeaks(xrdDataDirectory, xrdPeakDirectory, plotsDirectory, xLimit)


In [ ]:
list = ['mp-966800', 'mp-23202', 'mp-30930', 'mp-571260']
pXRDGenerator(list, "/Users/yasinbadawy/Library/CloudStorage/Box-Box/Data analysis/XRD/240219", "vSc5azvtxSkiFL7G", intensityThreshold=0)
xrdPlotter("/Users/yasinbadawy/Library/CloudStorage/Box-Box/Data analysis/XRD/240219", [20,80])